# Neural network to detect ISH expression
## First, import module

In [85]:
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.io import imread, imsave 
from keras.layers import Input, ZeroPadding2D, Conv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from keras.models import Model
from image_prep import read_pos
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Read traing data

In [86]:
session_list = {"71724696","73818754"}
m, n, c = 400,552,3
nfile = 0
for ses in session_list:
    input_dir = "C:/Users/bmesh/OneDrive - University of Southern California/allen/" + ses + "_input_resize"
    image_list = os.listdir(input_dir)
    nfile = nfile + len(image_list)
    
X_train = np.zeros((nfile, m, n, c))
count = 0
for ses in session_list:
    input_dir = "C:/Users/bmesh/OneDrive - University of Southern California/allen/" + ses + "_input_resize"
    image_list = os.listdir(input_dir)
    for im in image_list:
        X_train[count,:,:,:] = imread(os.path.join(input_dir,im))/255
        count = count + 1

In [87]:
# Version 1: use output_image as Y_train
count = 0
Y_train = np.zeros((nfile, m*n))
for ses in session_list:
    output_dir = "C:/Users/bmesh/OneDrive - University of Southern California/allen/" + ses + "_output_resize"
    image_list = os.listdir(output_dir)
for im in image_list:
    tmp = np.mean(imread(os.path.join(output_dir,im)),axis = 2)
    Y_train[count,:] = tmp.flatten()/255
    count = count + 1

In [ ]:
# Version 2: use detected postion as Y_train
count = 0
Y_train = np.zeros((nfile, m*n))
for ses in session_list:
    output_dir = "C:/Users/bmesh/OneDrive - University of Southern California/allen/" + ses + "_output_resize"
    image_list = os.listdir(output_dir)
for im in image_list:
    image = imread(os.path.join(output_dir,im))
    pos = read_pos(image, 127)
    Y_train[count,:] = tmp.flatten()/255
    count = count + 1

In [88]:
print("Shape of input size:" + str(X_train.shape))
print("Shape of output size:" + str(Y_train.shape))

Shape of input size:(135, 400, 552, 3)
Shape of output size:(135, 220800)


In [91]:
def DetectionModel(input_shape):
    """
    implement of CNN model for ISH expression pattern detection
    
    Arguments:
    input_shape -- shape of the image of the dataset
    (height, width, channel) as a tuple.
    Note that it does not include the "batch" as a dimenson
    if you have a batch like "X_train"
    then you can provide input_shape as "X_train.shape[1:]"
    
    Returns:
    model -- a Model() instance in Keras
    
    """
    # input place holder
    X_input = Input(input_shape)
    
    # zero-padding
    X = ZeroPadding2D((3,3))(X_input)
    
    # CONV -- BN -- RELU
    X = Conv2D(32, (7,7), strides = (1,1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)
    
    # MAXPOOL
    X = MaxPooling2D((2,2), name='max_pool')(X)
    
    # FLATTEN and FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(220800, activation = 'sigmoid', name='fc')(X)
    
    # CREATE MODEL
    model = Model(inputs = X_input, outputs = X, name = 'DetectionModel') 
    
    return model

In [92]:
detectionModel = DetectionModel(X_train.shape[1:])

ResourceExhaustedError: OOM when allocating tensor with shape[1766400,220800] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:RandomUniform]

In [74]:
detectionModel.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mean_squared_error'])

In [75]:
detectionModel.fit(x = X_train, y = Y_train, epochs = 1, batch_size = 1)

ValueError: in user code:

    C:\Users\bmesh\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\bmesh\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\bmesh\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\bmesh\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\bmesh\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\bmesh